In [1]:
# pricing an American put option - Longstaff-Schwarz
# option based on average price of two stocks 

import numpy as np 
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
# parameters of the stocks and the option 
S10 = 40.0
S20 = 40.0 
K = 40.0 
r = 0.06
sigma1 = 0.4
sigma2 = 0.2
rho = -0.5 
T = 2.0

In [3]:
# numeric parameters - number of intervals, number of samples, random numbers 
N = 500
h = T/N 
M = 50000

In [4]:
S1 = S10*np.ones([M,N+1])
S2 = S20*np.ones([M,N+1])
S3 = S10*np.ones([M,N+1])
Z1 = npr.randn(M,N)
Z2 = npr.randn(M,N)
b = 3                    # number of basis functions
disc = np.exp(-r*h)      # one period discount factor 

In [5]:
# generate price paths using the GBM formula
for n in range(0,N):       
    S1[:,n+1]=S1[:,n]*np.exp((r-sigma1**2/2)*h+(Z1[:,n])                              *sigma1*np.sqrt(h))
    S2[:,n+1]=S2[:,n]*np.exp((r-sigma2**2/2)*h+(rho*Z1[:,n]+np.sqrt(1-rho**2)*Z2[:,n])*sigma2*np.sqrt(h))


In [6]:
for i in range(0,M): 
    for j in range(0,N+1):  
        S3[i,j]=np.min([(S1[i,j],S2[i,j])])

In [7]:
S1[M-1,N], S2[M-1,N], S3[M-1,N]

(9.262132968881724, 42.55871420630111, 9.262132968881724)

In [8]:
# exercise value
def exer(SS):
    return((K-SS)*(K>SS))

# Basis functions
def psi(i,SS):
    if i==0:
        return(1)
    elif i==1:
        return((1-SS/K))
    elif i==2:
        return((1-SS/K)**2)
    elif i==3:
        return((1-SS/K)**3)

In [9]:
c = np.zeros([b,N+1])     # coefficients of continuation values
MM = np.zeros([b,b])      # normalization matrix
val = np.zeros([M,N+1])   # option values in simulation 
val[:,N] = exer(S3[:,N])   # option values at final time

In [10]:
for k in range(N-1,0,-1):
    sel = S3[:,k]<K
    val[~sel,k] =  disc*val[~sel,k+1]
    Ssel = S3[sel,k]
    valsel = disc*val[sel,k+1]
    for i in range(0,b):
        c[i,k] = np.mean( valsel *  psi(i,Ssel))      # was called d in file 
        MM[i,i] = np.mean( psi(i,Ssel) *  psi(i,Ssel) )   
        for j in range(i+1,b):
            MM[i,j] = np.mean( psi(i,Ssel) *  psi(j,Ssel) ) 
            MM[j,i] = MM[i,j] 
    c[:,k] = np.linalg.solve(MM,c[:,k])
    cont = np.zeros(Ssel.size)
    for i in range(0,b):
        cont = cont + c[i,k] * psi(i,Ssel)
    val[sel,k] = exer(Ssel) * (exer(Ssel) > cont) + disc*val[sel,k+1] * (exer(Ssel) < cont)
val[:,0] =  disc*val[:,1]
V1=np.mean(val[:,0])
e1=np.std(val[:,0])/np.sqrt(M)
print("price of the option is: {:.3f}, with a stochastic error of {:.5f}".format(V1,e1))

price of the option is: 8.749, with a stochastic error of 0.02728


In [11]:
# generate a further set of paths to get lower bound on option price 
M2 = 50000
Sq1 = S10*np.ones([M2,N+1])
Sq2 = S20*np.ones([M2,N+1])
Sq3 = S10*np.ones([M2,N+1])
Zq1 = npr.randn(M2,N)
Zq2 = npr.randn(M2,N)
for n in range(0,N):    
    Sq1[:,n+1]=Sq1[:,n]*np.exp((r-sigma1**2/2)*h+(Zq1[:,n])*sigma1*np.sqrt(h))
    Sq2[:,n+1]=Sq2[:,n]*np.exp((r-sigma2**2/2)*h+(rho*Zq1[:,n]+np.sqrt(1-rho**2)*Zq2[:,n])*sigma2*np.sqrt(h))  
for i in range(0,M): 
    for j in range(0,N+1):  
        Sq3[i,j]=np.min([(Sq1[i,j],Sq2[i,j])])
val2 = np.zeros(M2)
for n in range(1,N+1): 
    tem1 = exer( Sq3[:,n] )
    tem2 = np.zeros(M2)
    for i in range(0,b):
        tem2 = tem2 + c[i,n] * psi(i,Sq3[:,n])
    val2 = val2 + np.exp(-r*n*h)*(val2==0)*(tem1>tem2)*tem1
V2=np.mean(val2)
e2=np.std(val2)/np.sqrt(M2)
print("price of the option is: {:.3f}, with a stochastic error of {:.5f}".format(V2,e2))

price of the option is: 8.728, with a stochastic error of 0.02729
